In [ ]:
!pip install theano tensor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 909.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.7 MB/s eta 0:00:00
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668109 sha256=d5afa3163d1eed3d1965a619fa0

In [ ]:
import numpy as np
import theano
import theano.tensor as T
from keras.datasets import mnist

In [ ]:
# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], -1).astype('float32') / 255.0
X_test = X_test.reshape(X_test.shape[0], -1).astype('float32') / 255.0

# Define Theano symbolic variables
X_theano = T.matrix('X')

In [ ]:
input_dim = X_train.shape[1]
hidden_dim = 256
latent_dim = 2
W_enc_mean = theano.shared(np.random.randn(input_dim, hidden_dim), name='W_enc_mean')
b_enc_mean = theano.shared(np.zeros((hidden_dim,)), name='b_enc_mean')
W_enc_logvar = theano.shared(np.random.randn(input_dim, hidden_dim), name='W_enc_logvar')
b_enc_logvar = theano.shared(np.zeros((hidden_dim,)), name='b_enc_logvar')
W_dec = theano.shared(np.random.randn(hidden_dim, input_dim), name='W_dec')
b_dec = theano.shared(np.zeros((input_dim,)), name='b_dec')

In [ ]:
enc_hidden = T.tanh(T.dot(X_theano, W_enc_mean) + b_enc_mean)
enc_mean = T.dot(enc_hidden, W_enc_mean) + b_enc_mean
enc_logvar = T.dot(enc_hidden, W_enc_logvar) + b_enc_logvar

In [ ]:
epsilon = theano.shared(np.random.randn(input_dim, hidden_dim), name='epsilon')
latent_z = enc_mean + T.exp(0.5 * enc_logvar) * epsilon

In [ ]:
dec_hidden = T.tanh(T.dot(latent_z, W_dec) + b_dec)
dec_output = T.nnet.sigmoid(T.dot(dec_hidden, W_dec) + b_dec)

In [ ]:
# Define the reconstruction loss
recon_loss = -T.mean(T.sum(X_theano * T.log(dec_output) + (1 - X_theano) * T.log(1 - dec_output), axis=1))

In [ ]:
# Define the KL divergence
kl_divergence = -0.5 * T.mean(1 + enc_logvar - T.square(enc_mean) - T.exp(enc_logvar))

In [ ]:
# Define the total loss
total_loss = recon_loss + kl_divergence

In [ ]:
params = [W_enc_mean, b_enc_mean, W_enc_logvar, b_enc_logvar, W_dec, b_dec]
grads = T.grad(total_loss, params)

In [ ]:
updates = [(param, param - 0.01 * grad) for param, grad in zip(params, grads)]
train = theano.function(inputs=[X_theano], outputs=total_loss, updates=updates)

In [ ]:
batch_size = 128
num_batches = X_train.shape[0]
epochs = 10
for epoch in range(epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size
        train(X_train[start:end])

In [ ]:
# Generate new images
latent_samples = np.random.randn(10, latent_dim)
generated_images = theano.function(inputs=[latent_z], outputs=dec_output)
generated_images = generated_images(latent_samples)